In [ ]:
!pip install pyspark
!pip install sparksql-magic
!mkdir -p $HOME/spark_jars
!curl -L -o $HOME/spark_jars/hadoop-aws-3.3.1.jar https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar
!curl -L -o $HOME/spark_jars/aws-java-sdk-bundle-1.11.901.jar https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.901/aws-java-sdk-bundle-1.11.901.jar

In [10]:
%load_ext sparksql_magic

In [11]:
from pyspark.sql import SparkSession
import pyspark
import os

jars_dir = os.path.expanduser("~/spark_jars")

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars', f'{jars_dir}/hadoop-aws-3.3.1.jar,{jars_dir}/aws-java-sdk-bundle-1.11.901.jar')
        # Packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2')
        # SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
        # Configuring Catalog
        .set('spark.sql.catalog.icebergcat', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.icebergcat.type', 'hadoop')
        .set('spark.sql.catalog.icebergcat.warehouse', 's3a://iceberg-warehouse')
        # MinIO specific configurations
        .set('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000')
        .set('spark.hadoop.fs.s3a.access.key', 'yr2B94hM8RwzKCeqaDbx')  # Replace with your MinIO access key
        .set('spark.hadoop.fs.s3a.secret.key', 'cPA8KzD5cV41U0tXoeKu0mQmAQwSZabOMfOrZg2b')  # Replace with your MinIO secret key
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
        .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Spark Running


In [24]:
%%sparksql

CREATE TABLE IF NOT EXISTS icebergcat.nyc.taxis
(
  vendor_id bigint,
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id);

In [26]:
%%sparksql

INSERT INTO icebergcat.nyc.taxis
VALUES (1, 1000371, 1.8, 15.32, 'N'), (2, 1000372, 2.5, 22.15, 'N'), (2, 1000373, 0.9, 9.01, 'N'), (1, 1000374, 8.4, 42.13, 'Y');

In [28]:
%%sparksql -c -v df

SELECT * FROM icebergcat.nyc.taxis;

cache dataframe with lazy load
create temporary view `df`


vendor_id,trip_id,trip_distance,fare_amount,store_and_fwd_flag
1,1000371,1.7999999523162842,15.32,N
1,1000374,8.399999618530273,42.13,Y
2,1000372,2.5,22.15,N
2,1000373,0.8999999761581421,9.01,N
